In [1]:
import findspark

findspark.init(spark_home='/Users/zhanghousu/spark/spark-3.3.1-bin-hadoop2',
                python_path='/Users/zhanghousu/conda/bin/python')

from pyspark import SparkContext
sc = SparkContext()

23/04/29 15:51:38 WARN Utils: Your hostname, MacBook-Pro-279.local resolves to a loopback address: 127.0.0.1; using 10.79.140.225 instead (on interface en0)
23/04/29 15:51:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/29 15:51:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/29 15:51:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/29 15:51:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/29 15:51:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/04/29 15:51:40 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [2]:
# read file
lines = sc.textFile("movie/ratings.csv")

header = lines.first()
lines = lines.filter(lambda line: line != header)

# extract (movieId, rating)
movie_ratings = lines.map(lambda x: (int(x.split(',')[1]), float(x.split(',')[2])))

# group by movieId
grouped_ratings = movie_ratings.groupByKey()

# calculate average rating
avg_ratings = grouped_ratings.mapValues(lambda x: sum(x) / len(x))

# order by average rating
sorted_avg_ratings = avg_ratings.sortBy(lambda x: x[1], ascending=False)

# print top 10
results = sorted_avg_ratings.collect()

# for result in results:
#     print("MovieId: {}, Avg Rating: {:.2f}".format(result[0], result[1]))


In [3]:
perfect_ratings = avg_ratings.filter(lambda x: x[1] >= 3.5)

# Collect results and print
five_score = perfect_ratings.collect()


In [4]:
# for result in five_score:
from pyspark.sql import SparkSession
spark = SparkSession(sc)
perfect_ratings_df = spark.createDataFrame(five_score, schema=["movieId", "avg_rating"])
perfect_ratings_df.show(5)

+-------+------------------+
|movieId|        avg_rating|
+-------+------------------+
|    110| 4.016057252826558|
|    858| 4.339810758717364|
|  91542| 3.738952195664258|
| 112552| 4.120283855706682|
|   1210|3.9896115699843735|
+-------+------------------+
only showing top 5 rows



In [5]:
# read file
movi_df = spark.read.csv('movie/movies_metadata.csv', header=True, escape='"')
movi_df.show(10)

+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+--------------------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage|   id|  imdb_id|original_language|      original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|               title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+-----+---------+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+-----------

In [6]:
movi_df = movi_df["id","genres"]
movi_df.show(5)

+-----+--------------------+
|   id|              genres|
+-----+--------------------+
|  862|[{'id': 16, 'name...|
| 8844|[{'id': 12, 'name...|
|15602|[{'id': 10749, 'n...|
|31357|[{'id': 35, 'name...|
|11862|[{'id': 35, 'name...|
+-----+--------------------+
only showing top 5 rows



In [7]:
movi_df = movi_df.withColumnRenamed("id", "movieId")
movi_df = movi_df.drop("id")
movi_df.show(5)

+-------+--------------------+
|movieId|              genres|
+-------+--------------------+
|    862|[{'id': 16, 'name...|
|   8844|[{'id': 12, 'name...|
|  15602|[{'id': 10749, 'n...|
|  31357|[{'id': 35, 'name...|
|  11862|[{'id': 35, 'name...|
+-------+--------------------+
only showing top 5 rows



In [8]:
from pyspark.sql.functions import from_json, col, explode, \
    get_json_object, regexp_replace, array
from pyspark.sql.types import StructType, StructField, \
    StringType, IntegerType, ArrayType, MapType, DoubleType

json_schema = ArrayType(StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True)
]))

mv_ges = movi_df.withColumn('genres', regexp_replace("genres", 'None', "'None'"))
mv_ges = mv_ges.withColumn("genres_array", from_json(col("genres"), json_schema))

# explode
mv_ges = mv_ges.select(col("movieId"), explode(col("genres_array")).alias("genres_item"))

# extract
mv_ges_df = mv_ges.withColumn("genre_name", col("genres_item")["name"]) \
    .drop('genres_item')

mv_ges_df.show(5)


+-------+----------+
|movieId|genre_name|
+-------+----------+
|    862| Animation|
|    862|    Comedy|
|    862|    Family|
|   8844| Adventure|
|   8844|   Fantasy|
+-------+----------+
only showing top 5 rows



In [13]:
# join two dataframe
joined_df = perfect_ratings_df.join(mv_ges_df, on="movieId", how="inner")
joined_df = joined_df.drop("avg_rating")
joined_df.show(5)


+-------+----------+
|movieId|genre_name|
+-------+----------+
|    110|   Romance|
|    110|   Mystery|
|    110|     Drama|
|    858|   Romance|
|    858|     Drama|
+-------+----------+
only showing top 5 rows



In [28]:
%pip install wheel setuptools --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: wheel
    Found existing installation: wheel 0.37.1
    Uninstalling wheel-0.37.1:
      Successfully uninstalled wheel-0.37.1
  Attempting uninstall: setuptools
    Found existing installation: setuptools 63.4.1
    Uninstalling setuptools-63.4.1:
      Successfully uninstalled setuptools-63.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [33]:
%pip install wordcloud==1.8.1 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [121 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-11.1-arm64-cpython-39
      creating build/lib.macosx-11.1-arm64-cpython-39/wordcloud
      copying wordcloud/wordcloud_cli.py -> build/lib.macosx-11.1-arm64-cpython-39/wordcloud
      copying wordcloud/_version.py -> build/lib.macosx-11.1-arm64-cpython-39/wordcloud
      copying wordcloud/__init__.py -> build/lib.macosx-11.1-arm64-cpython-39/wordcloud
      copying wordcloud/tokenization.py -> build/lib.macosx-11.1-arm64-cpython-39/wordcloud
      copying wordcloud/wordcloud.py -> build/lib.macosx-11.1-arm64-cpython-39/wordcloud
      copying wordcloud/color_from_image.py -> build/lib.macosx-11.1-arm64-c

In [23]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# 将DataFrame的genre_name列转换为一个Python列表
genre_name_list = joined_df.select("genre_name").rdd.flatMap(lambda x: x).collect()

# 将列表转换为单个字符串，用空格分隔每个genre_name
genre_name_text = " ".join(genre_name_list)

# 生成词云
wordcloud = WordCloud(background_color="white", width=800, height=800, margin=2).generate(genre_name_text)

# 显示词云
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()


ModuleNotFoundError: No module named 'wordcloud'